# Neural Network Regression task - Bike sharing

Bike sharing systems are new generation of traditional bike rentals where whole process from membership, rental and return has become automatic. Through these systems, a user is able to easily rent a bike from a particular position and return it at another place.

The dataset contains the hourly count of rental bikes between years 2011 and 2012 in the Capital Bikeshare system (Wasington DC) with the corresponding weather and seasonal information.

The goal of this task is to train a regressor to predict total counts of bike rentals based on the provided features for a given hour. 

## Data source
[http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset)

## Feature description
* **dteday** - date time stamot
* **season** - season (1: spring, 2: summer, 3: fall, 4: winter)
* **yr** - year (0: 2011, 1: 2012)
* **mnth** - month (1 to 12)
* **hr** - hour (0 to 23)
* **holiday** - 1 if the day is a holiday, else 0 (extracted from [holiday schedules](https://dchr.dc.gov/page/holiday-schedules))
* **weekday** - day of the week (0 to 6)
* **workingday** - is 1 if day is neither weekend nor holiday, else 0.
* **weathersit** 
    * 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    * 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    * 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    * 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
* **temp** - Normalized temperature in degrees of Celsius.
* **atemp** - Normalized feeling temperature in degrees Celsius.
* **hum** - Normalized relative humidity.
* **windspeed** - Normalized wind speed.
* **casual** - Count of casual users.
* **registered** - Count of registered users.
* **cnt** -  Count of total rental bikes including both casual and registered. This is the target value. 

In [1]:
import pandas as pd
data = pd.read_csv('../data/bikes.csv', sep=',')
data.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Add some features from the past

Since we have time stamp of every measurement, we can see the data as a time series and use data from the past. We add one feature column computed from the data of the previous hour.

In [2]:
data.sort_values(['dteday', 'hr'])
cnt = data['cnt']
data['hist'] = cnt.shift(1)
data = data[1:]
data.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,hist
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40,16.0
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32,40.0
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13,32.0
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1,13.0
5,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1,1.0


## Neural Network regressor

Implement a neural network regressor based on all reasonable features from the input data set. Notice that some of the features from the input data cannot be used.

### Data preparation

Prepare train and test data sets.

In [3]:
from sklearn.model_selection import train_test_split

X_all = data[['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit','temp', 'atemp', 'hum', 'windspeed', 'hist']]
y_all = data['cnt']


X_train, X_test, y_train, y_test = train_test_split(
    X_all, 
    y_all,
    random_state=1,
    test_size=0.2)

print('Train size: {}'.format(len(X_train)))
print('Test size: {}'.format(len(X_test)))

Train size: 13902
Test size: 3476


Standardize the features

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Training a regressor
Design and train a regression model. Use the [mean squared error](https://keras.io/losses/) loss function. Experiment with various architectures, [activation functions](https://keras.io/activations/) and [optimizers](https://keras.io/optimizers/).

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential()

model.add(Dense(32, input_shape=(13, )))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(32))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('linear'))

Using TensorFlow backend.


Compile the model

In [6]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mae'])

Train the model

In [7]:
model.fit(X_train, y_train,
          batch_size = 128, epochs = 500, verbose=1,
          validation_data=(X_test, y_test))

Train on 13902 samples, validate on 3476 samples
Epoch 1/500
13902/13902 [==============================] - 0s 31us/step - loss: 67972.2543 - mean_absolute_error: 188.6025 - val_loss: 67042.7090 - val_mean_absolute_error: 188.2367
Epoch 2/500
13902/13902 [==============================] - 0s 14us/step - loss: 65642.4720 - mean_absolute_error: 185.5825 - val_loss: 64454.9098 - val_mean_absolute_error: 182.7684
Epoch 3/500
13902/13902 [==============================] - 0s 14us/step - loss: 62967.3105 - mean_absolute_error: 177.3273 - val_loss: 61975.2877 - val_mean_absolute_error: 174.1439
Epoch 4/500
13902/13902 [==============================] - 0s 14us/step - loss: 60927.3228 - mean_absolute_error: 172.0863 - val_loss: 60377.9270 - val_mean_absolute_error: 171.0329
Epoch 5/500
13902/13902 [==============================] - 0s 15us/step - loss: 59424.2161 - mean_absolute_error: 169.1011 - val_loss: 59013.6758 - val_mean_absolute_error: 168.5265
Epoch 6/500
13902/13902 [================

13902/13902 [==============================] - 0s 12us/step - loss: 29098.1309 - mean_absolute_error: 101.6079 - val_loss: 28257.2013 - val_mean_absolute_error: 96.6441
Epoch 46/500
13902/13902 [==============================] - 0s 12us/step - loss: 28605.2977 - mean_absolute_error: 100.7987 - val_loss: 27760.1359 - val_mean_absolute_error: 95.6701
Epoch 47/500
13902/13902 [==============================] - 0s 12us/step - loss: 28111.8800 - mean_absolute_error: 99.9229 - val_loss: 27274.0128 - val_mean_absolute_error: 94.4110
Epoch 48/500
13902/13902 [==============================] - 0s 16us/step - loss: 27678.0097 - mean_absolute_error: 98.8663 - val_loss: 26797.6376 - val_mean_absolute_error: 93.2904
Epoch 49/500
13902/13902 [==============================] - 0s 15us/step - loss: 27221.1988 - mean_absolute_error: 97.9356 - val_loss: 26332.1942 - val_mean_absolute_error: 92.3054
Epoch 50/500
13902/13902 [==============================] - 0s 14us/step - loss: 26889.0341 - mean_absolut

13902/13902 [==============================] - 0s 15us/step - loss: 16324.6262 - mean_absolute_error: 78.6879 - val_loss: 14151.7601 - val_mean_absolute_error: 66.8608
Epoch 91/500
13902/13902 [==============================] - 0s 13us/step - loss: 16050.5301 - mean_absolute_error: 78.3382 - val_loss: 13961.0820 - val_mean_absolute_error: 66.2768
Epoch 92/500
13902/13902 [==============================] - 0s 12us/step - loss: 16028.2127 - mean_absolute_error: 78.3540 - val_loss: 13788.4309 - val_mean_absolute_error: 65.7278
Epoch 93/500
13902/13902 [==============================] - 0s 12us/step - loss: 15807.9920 - mean_absolute_error: 78.0520 - val_loss: 13622.2794 - val_mean_absolute_error: 65.5657
Epoch 94/500
13902/13902 [==============================] - 0s 15us/step - loss: 15686.3968 - mean_absolute_error: 77.7628 - val_loss: 13461.2461 - val_mean_absolute_error: 65.2997
Epoch 95/500
13902/13902 [==============================] - 0s 15us/step - loss: 15529.9823 - mean_absolute_

13902/13902 [==============================] - 0s 15us/step - loss: 12504.3953 - mean_absolute_error: 73.5954 - val_loss: 9620.1918 - val_mean_absolute_error: 57.2119
Epoch 136/500
13902/13902 [==============================] - 0s 17us/step - loss: 12515.7387 - mean_absolute_error: 73.4900 - val_loss: 9585.0133 - val_mean_absolute_error: 57.6344
Epoch 137/500
13902/13902 [==============================] - 0s 16us/step - loss: 12207.6742 - mean_absolute_error: 72.7395 - val_loss: 9563.3396 - val_mean_absolute_error: 56.7696
Epoch 138/500
13902/13902 [==============================] - 0s 14us/step - loss: 12403.3162 - mean_absolute_error: 73.1400 - val_loss: 9496.2406 - val_mean_absolute_error: 57.1549
Epoch 139/500
13902/13902 [==============================] - 0s 15us/step - loss: 12190.2102 - mean_absolute_error: 72.8092 - val_loss: 9448.8539 - val_mean_absolute_error: 57.6047
Epoch 140/500
13902/13902 [==============================] - 0s 14us/step - loss: 12409.6298 - mean_absolute_

13902/13902 [==============================] - 0s 15us/step - loss: 11397.3879 - mean_absolute_error: 71.5982 - val_loss: 8439.4275 - val_mean_absolute_error: 55.3149
Epoch 181/500
13902/13902 [==============================] - 0s 14us/step - loss: 11448.1627 - mean_absolute_error: 71.7694 - val_loss: 8462.4036 - val_mean_absolute_error: 54.5344
Epoch 182/500
13902/13902 [==============================] - 0s 15us/step - loss: 11170.0618 - mean_absolute_error: 70.7141 - val_loss: 8434.2530 - val_mean_absolute_error: 55.6575
Epoch 183/500
13902/13902 [==============================] - 0s 14us/step - loss: 11215.8261 - mean_absolute_error: 70.9982 - val_loss: 8405.8028 - val_mean_absolute_error: 54.6580
Epoch 184/500
13902/13902 [==============================] - 0s 13us/step - loss: 11430.4573 - mean_absolute_error: 71.7006 - val_loss: 8391.5491 - val_mean_absolute_error: 55.9180
Epoch 185/500
13902/13902 [==============================] - 0s 13us/step - loss: 11324.8236 - mean_absolute_

...

13902/13902 [==============================] - 0s 14us/step - loss: 10250.7819 - mean_absolute_error: 67.7904 - val_loss: 7779.2935 - val_mean_absolute_error: 54.3825
Epoch 316/500
13902/13902 [==============================] - 0s 12us/step - loss: 10613.3903 - mean_absolute_error: 68.9100 - val_loss: 7757.9260 - val_mean_absolute_error: 54.1682
Epoch 317/500
13902/13902 [==============================] - 0s 12us/step - loss: 10581.4082 - mean_absolute_error: 68.6028 - val_loss: 7774.9045 - val_mean_absolute_error: 53.3482
Epoch 318/500
13902/13902 [==============================] - 0s 12us/step - loss: 10505.0052 - mean_absolute_error: 68.4918 - val_loss: 7785.9774 - val_mean_absolute_error: 54.1384
Epoch 319/500
13902/13902 [==============================] - 0s 12us/step - loss: 10571.1104 - mean_absolute_error: 68.5557 - val_loss: 7781.5668 - val_mean_absolute_error: 53.6719
Epoch 320/500
13902/13902 [==============================] - 0s 13us/step - loss: 10360.1965 - mean_absolute_

13902/13902 [==============================] - 0s 13us/step - loss: 10301.7546 - mean_absolute_error: 67.9778 - val_loss: 7775.4707 - val_mean_absolute_error: 53.2785
Epoch 361/500
13902/13902 [==============================] - 0s 12us/step - loss: 10523.7941 - mean_absolute_error: 68.2510 - val_loss: 7658.8389 - val_mean_absolute_error: 54.4650
Epoch 362/500
13902/13902 [==============================] - 0s 12us/step - loss: 10315.0256 - mean_absolute_error: 67.8006 - val_loss: 7753.7744 - val_mean_absolute_error: 52.5492
Epoch 363/500
13902/13902 [==============================] - 0s 12us/step - loss: 10555.7404 - mean_absolute_error: 68.3840 - val_loss: 7717.7925 - val_mean_absolute_error: 54.1469
Epoch 364/500
13902/13902 [==============================] - 0s 12us/step - loss: 10284.1278 - mean_absolute_error: 67.7637 - val_loss: 7749.0210 - val_mean_absolute_error: 53.1039
Epoch 365/500
13902/13902 [==============================] - 0s 12us/step - loss: 10187.7889 - mean_absolute_

13902/13902 [==============================] - 0s 13us/step - loss: 10083.2352 - mean_absolute_error: 67.4443 - val_loss: 7631.1000 - val_mean_absolute_error: 52.9085
Epoch 406/500
13902/13902 [==============================] - 0s 13us/step - loss: 10339.6673 - mean_absolute_error: 67.5054 - val_loss: 7662.9939 - val_mean_absolute_error: 53.1005
Epoch 407/500
13902/13902 [==============================] - 0s 12us/step - loss: 10292.7103 - mean_absolute_error: 67.7906 - val_loss: 7634.1472 - val_mean_absolute_error: 53.4277
Epoch 408/500
13902/13902 [==============================] - 0s 12us/step - loss: 10302.8310 - mean_absolute_error: 67.5504 - val_loss: 7677.1550 - val_mean_absolute_error: 52.7807
Epoch 409/500
13902/13902 [==============================] - 0s 12us/step - loss: 10356.4823 - mean_absolute_error: 67.4452 - val_loss: 7641.5236 - val_mean_absolute_error: 54.0651
Epoch 410/500
13902/13902 [==============================] - 0s 12us/step - loss: 10373.0021 - mean_absolute_

13902/13902 [==============================] - 0s 13us/step - loss: 10056.0359 - mean_absolute_error: 66.6271 - val_loss: 7579.6897 - val_mean_absolute_error: 53.8372
Epoch 451/500
13902/13902 [==============================] - 0s 12us/step - loss: 10200.1661 - mean_absolute_error: 67.3370 - val_loss: 7549.0839 - val_mean_absolute_error: 53.4191
Epoch 452/500
13902/13902 [==============================] - 0s 13us/step - loss: 10249.7210 - mean_absolute_error: 67.2606 - val_loss: 7587.2115 - val_mean_absolute_error: 54.3568
Epoch 453/500
13902/13902 [==============================] - 0s 12us/step - loss: 10301.0829 - mean_absolute_error: 67.5148 - val_loss: 7617.3003 - val_mean_absolute_error: 53.2685
Epoch 454/500
13902/13902 [==============================] - 0s 14us/step - loss: 10090.3730 - mean_absolute_error: 66.9598 - val_loss: 7606.9064 - val_mean_absolute_error: 53.5483
Epoch 455/500
13902/13902 [==============================] - 0s 13us/step - loss: 10240.2032 - mean_absolute_

13902/13902 [==============================] - 0s 15us/step - loss: 10212.8652 - mean_absolute_error: 67.1754 - val_loss: 7633.5725 - val_mean_absolute_error: 52.7917
Epoch 496/500
13902/13902 [==============================] - 0s 12us/step - loss: 10076.3063 - mean_absolute_error: 66.7001 - val_loss: 7557.8994 - val_mean_absolute_error: 53.2178
Epoch 497/500
13902/13902 [==============================] - 0s 14us/step - loss: 10172.8539 - mean_absolute_error: 66.7939 - val_loss: 7553.3128 - val_mean_absolute_error: 52.8811
Epoch 498/500
13902/13902 [==============================] - 0s 14us/step - loss: 10143.6863 - mean_absolute_error: 66.8158 - val_loss: 7592.2043 - val_mean_absolute_error: 52.5961
Epoch 499/500
13902/13902 [==============================] - 0s 14us/step - loss: 9822.1817 - mean_absolute_error: 66.2332 - val_loss: 7556.2112 - val_mean_absolute_error: 54.0937
Epoch 500/500
13902/13902 [==============================] - 0s 14us/step - loss: 9911.8234 - mean_absolute_er

### Evaluate the models

Measure mean squared error and mean absolute error evaluation metrics on both train and test data sets. Compute the mean and standard deviation of the target values.

In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

y_pred = model.predict(X_test)
print ("Test mean: {}, std: {}".format(np.mean(y_test), np.std(y_test)))
print("Test Root mean squared error: {:.2f}".format(np.sqrt(mean_squared_error(y_test, y_pred))))
print("Test Mean absolute error: {:.2f}".format(mean_absolute_error(y_test, y_pred)))

Test mean: 190.1343498273878, std: 181.50427424823098
Test Root mean squared error: 87.18
Test Mean absolute error: 54.90


In [9]:
y_pred = model.predict(X_train)
print("Train Root mean squared error: %.2f"
      % np.sqrt(mean_squared_error(y_train, y_pred)))
print("Train Mean absolute error: %.2f"
      % mean_absolute_error(y_train, y_pred))

Train Root mean squared error: 87.68
Train Mean absolute error: 54.72
